In [1]:
import pandas as pd
from statsmodels.distributions.empirical_distribution import ECDF

In [2]:
# Extract the new sigmas from the UNC232 PAM50 training set. 
# UNC232 gene expression matrix and Immunohistochemistry (IHC) classes are provided.
gene_expresion = pd.read_csv('./Subgroup-AFM/UNC232_gene_expresion.txt', sep='\t', index_col=0)
groups = pd.read_csv('./Subgroup-AFM/UNC232_groups.txt', sep='\t', index_col=0)

In [3]:
gene_expresion.head()

,Normal_Breast_10,Normal_Breast_2,Normal_Breast_3,Normal_Breast_4_Custom,Normal_Breast_7,Normal_Breast_9_Custom,Normal_Str574_100%,normalbreast_BR00_0572A,normalbreast_BR00_0587A,normalbreast_BR05_0865A,...,H1AUNC_1504C,H1AUNC_1505C,H1AUNC_1508_C,H1AUNC_1515_C,H1AUNC_1517_C,H1AUNC_1520_C,H1AUNC_1522_C,H1AUNC_1523_C,H1AUNC_1570_C,H1AUNC_1613_C
ACTR3B,-1.151,-0.485,0.298,1.153,-0.287,1.082,0.691,-0.483,-0.649,1.136,...,0.022,-1.588,-0.276,-0.694,-0.672,-0.761,-1.110,-1.331,-0.608,-0.697
ANLN,-3.736,-3.739,-2.848,-4.717,-3.681,-4.544,-4.386,-4.756,NaN,-3.354,...,-3.151,-2.703,NaN,NaN,-2.297,-3.603,-3.724,-3.852,-2.489,-3.635
BAG1,0.260,0.591,0.359,0.098,0.441,0.037,-0.141,-0.035,0.242,0.247,...,-0.643,0.286,2.204,0.024,-0.872,1.181,1.995,0.299,-1.065,0.362
BCL2,1.300,1.580,1.292,1.954,1.911,0.814,0.386,2.230,1.782,0.454,...,0.621,0.838,0.015,0.597,-0.209,0.352,0.035,0.503,0.243,0.743
BIRC5,-2.860,-3.250,-2.493,-3.237,-2.156,-3.807,-3.656,-4.554,-4.918,-3.747,...,-3.000,-2.524,-4.695,-3.235,-2.581,-2.778,-2.936,-2.534,-0.362,-3.456


In [4]:
groups.head()

,IHC_subgroup_v4.0
H1A6433_2608V2_Na,TNBC
H1A6433_2610V2_Na,TNBC
H1A6433_3010_V2,TNBC
H1A7411_0111_C,TNBC
H1A9529_0044Na_C,TNBC


In [5]:
pd.Series(groups.values.ravel()).dropna().unique()

array(['TNBC', 'ERpos_HER2neg', 'HER2pos_ERneg', 'HER2pos_ERpos'],
      dtype=object)

In [6]:
def get_sigma(gene_expresion, groups):
    """Build quantile dataframe for each IHC group.
    
    For each gene it calculates the ECDF function within each IHC group,
    and calls it using the overall median of that gene.
    
    :param gene_expresion: Gene expresion dataframe, i.e. UNC232
    :param groups: IHC label dataframe for each sample on the gene expresion.
    
    :return: Quantile dataframe.
    """
    groups_cols = pd.Series(groups.values.ravel()).dropna().unique()
    res = pd.DataFrame({}, columns=groups_cols, index=gene_expresion.index)
    for name, values in gene_expresion.iterrows():
        for col in groups:
            unique = groups[col].dropna().unique()
            for u in unique:
                samples_from_group = groups.loc[groups[col] == u].index
                subset = values[samples_from_group].dropna()
                res[u][name] = ECDF(subset)(values.median())
    return res

In [7]:
sigma = get_sigma(gene_expresion, groups).sort_index(axis=1)

/var/folders/7y/y0qtnc214vlf39p5lk5ckc6w0000gn/T/ipykernel_72925/2591036951.py:20: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  res[u][name] = ECDF(subset)(values.median())
/var/folders/7y/y0qtnc214vlf39p5lk5ckc6w0000gn/T/ipykernel_72925/25

In [21]:
sigma.head

<bound method NDFrame.head of         ERpos_HER2neg HER2pos_ERneg HER2pos_ERpos      TNBC
ACTR3B       0.601852      0.833333      0.777778  0.175439
ANLN          0.68932      0.166667      0.411765  0.218182
BAG1          0.37963      0.916667      0.611111  0.666667
BCL2         0.564815      0.416667           0.5  0.526316
BIRC5        0.666667           0.2      0.352941  0.296296
BLVRA        0.342593      0.583333      0.277778  0.701754
CCNB1         0.62963      0.333333      0.444444   0.22807
CCNE1        0.722222      0.166667      0.444444  0.192982
CDC20        0.703704      0.166667      0.555556  0.245614
CDC6         0.705882          0.25      0.176471  0.245283
CDCA1        0.653846      0.333333      0.333333  0.214286
CDH3         0.740741          0.25      0.555556  0.303571
CENPF        0.694444      0.166667      0.388889  0.192982
CEP55        0.707547      0.166667      0.411765  0.175439
CXXC5        0.296296      0.583333      0.222222  0.842105
EGFR      

In [9]:
gene_expr = pd.read_csv('tables/nopdx/immune-log-normalized-counts.csv', sep=',', index_col=0)

In [11]:
gene_expr

,DSP-1001660005648-A-C05.dcc,DSP-1001660005648-A-C06.dcc,DSP-1001660005648-A-C12.dcc,DSP-1001660005648-A-D01.dcc,DSP-1001660005648-A-D05.dcc,DSP-1001660005648-A-D06.dcc,DSP-1001660005648-A-D09.dcc,DSP-1001660005648-A-D10.dcc,DSP-1001660005648-A-E06.dcc,DSP-1001660005649-B-B11.dcc,...,DSP-1001660012099-C-F02.dcc,DSP-1001660012099-C-F03.dcc,DSP-1001660012099-C-F10.dcc,DSP-1001660012099-C-F11.dcc,DSP-1001660012099-C-G01.dcc,DSP-1001660012099-C-G02.dcc,DSP-1001660012099-C-G04.dcc,DSP-1001660012099-C-G05.dcc,DSP-1001660012099-C-G06.dcc,DSP-1001660012099-C-G09.dcc
A2M,4.107242,4.107242,2.338757,3.719841,4.837890,0.000000,2.086422,4.810329,4.810329,3.478541,...,5.486509,3.788356,5.273800,4.180832,5.681659,3.478541,2.338757,4.664028,4.862871,5.913046
ACADM,1.599000,2.553459,2.338757,2.338757,0.000000,2.461956,0.000000,1.889777,0.000000,1.599000,...,3.054085,2.143336,1.963122,1.702576,2.740309,2.602489,0.000000,1.730852,1.169169,1.780394
APOE,6.170459,5.893479,4.267836,2.338757,3.188612,1.487410,2.086422,5.176926,3.563538,1.599000,...,6.129228,4.368244,5.475270,5.045197,6.930126,3.478541,7.360727,3.898734,5.143335,4.412330
ASS1,1.599000,2.553459,1.599000,3.478541,5.320600,3.768392,1.391475,2.680675,2.680675,1.599000,...,2.905710,3.696260,2.636517,3.768392,1.780394,3.018397,1.010541,3.361341,2.976091,2.740309
SERPING1,2.825379,3.425030,2.338757,2.338757,2.086422,2.680675,0.000000,3.860853,1.889777,3.926506,...,4.356257,2.276513,4.372501,1.487410,4.237115,3.826871,2.602489,3.361341,3.818914,3.311658
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WDR55,2.338757,0.000000,2.825379,2.338757,2.086422,0.927461,2.086422,1.889777,2.680675,2.338757,...,1.780394,2.398427,1.963122,1.702576,1.391475,1.010541,1.010541,1.730852,2.425156,0.000000
ZNF66,1.599000,0.000000,0.000000,3.188612,2.086422,2.204048,0.000000,0.000000,3.563538,2.825379,...,2.553459,1.437714,2.163131,2.338757,2.338757,2.825379,2.338757,2.163131,2.583828,2.086422
GLUD1,2.825379,3.188612,2.825379,2.338757,0.000000,2.680675,4.237115,1.889777,1.889777,1.599000,...,3.054085,3.317853,3.277560,4.068989,3.719841,2.825379,1.599000,2.765131,3.373809,3.628105
NDUFA3,2.338757,2.086422,2.825379,2.338757,3.425030,2.680675,2.086422,0.000000,1.889777,3.926506,...,2.905710,2.510838,2.992354,3.324704,2.553459,2.015783,2.825379,2.765131,2.976091,3.054085


In [19]:
pam50genes = pd.read_csv('tables/pam50-genes.csv', sep=',', index_col=0)
pam50genes = pam50genes.index
pam50genes

Index(['ACTR3B', 'ANLN', 'BAG1', 'BCL2', 'BIRC5', 'BLVRA', 'CCNB1', 'CCNE1',
       'CDC20', 'CDC6', 'CDCA1', 'CDH3', 'CENPF', 'CEP55', 'CXXC5', 'EGFR',
       'ERBB2', 'ESR1', 'EXO1', 'FGFR4', 'FOXA1', 'FOXC1', 'GPR160', 'GRB7',
       'KIF2C', 'KNTC2', 'KRT14', 'KRT17', 'KRT5', 'MAPT', 'MDM2', 'MELK',
       'MIA', 'MKI67', 'MLPH', 'MMP11', 'MYBL2', 'MYC', 'NAT1', 'ORC6L', 'PGR',
       'PHGDH', 'PTTG1', 'RRM2', 'SFRP1', 'SLC39A6', 'TMEM45B', 'TYMS',
       'UBE2C', 'UBE2T'],
      dtype='object')

In [ ]:
#### extract receptor status info from immune_metadata
#### use that to categorize the samples
#### normalize based on code